In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle


In [2]:
pickle_file = 'thumbnail_78_128_12Dec.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset=save['train_datasets'],
  train_labels=save['train_labels'],
  val_dataset=save['val_datasets'],
  val_labels=save['val_labels'],
  test_dataset=save['test_datasets'],
  test_images=save['test_images'],
  fish_names_train=save['fish_names_train']
  del save  
  f.close()
image_size_h = 78
image_size_w = 128
num_labels=8
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size_h * image_size_w)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset[0], train_labels[0])
valid_dataset, valid_labels = reformat(val_dataset[0], val_labels[0])
test_dataset, test_labels = reformat(test_dataset[0], val_labels[0])
test_images=test_images[0]

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset)
  tf_train_labels = tf.constant(train_labels)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size_h * image_size_w, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [32]:
num_steps = 801
probabilities=[]
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  probabilities=test_prediction.eval()

Initialized
Loss at step 0: 41.147240
Training accuracy: 12.3%
Validation accuracy: 45.0%
Loss at step 100: 15.314314
Training accuracy: 55.7%
Validation accuracy: 38.8%
Loss at step 200: 9.250088
Training accuracy: 62.6%
Validation accuracy: 47.6%
Loss at step 300: 1.150792
Training accuracy: 85.3%
Validation accuracy: 78.4%
Loss at step 400: 0.894319
Training accuracy: 88.7%
Validation accuracy: 82.1%
Loss at step 500: 3.097551
Training accuracy: 77.1%
Validation accuracy: 74.2%
Loss at step 600: 0.394604
Training accuracy: 94.1%
Validation accuracy: 84.3%
Loss at step 700: 0.278136
Training accuracy: 95.5%
Validation accuracy: 85.4%
Loss at step 800: 0.213001
Training accuracy: 96.4%
Validation accuracy: 85.7%


In [33]:
print(probabilities)

#print(session.run(test_prediction, feed_dict={x: tf_test_dataset}))

[[  1.67580186e-11   3.76413392e-13   6.79152208e-06 ...,   3.72520268e-01
    1.08758266e-10   4.47700505e-18]
 [  1.00000000e+00   3.63107185e-15   1.53113481e-17 ...,   1.28368227e-23
    2.58819191e-20   5.97696689e-17]
 [  9.98564780e-01   1.35787104e-07   1.97254979e-15 ...,   1.43513770e-03
    2.79747824e-16   3.85560117e-12]
 ..., 
 [  9.99999881e-01   9.69606830e-08   4.25342019e-11 ...,   1.24242716e-09
    2.73024577e-13   3.69362807e-09]
 [  9.99959230e-01   2.10512245e-13   9.76319828e-13 ...,   3.09014641e-12
    3.94418817e-15   1.33902049e-05]
 [  4.99089122e-01   6.06905496e-06   4.60289495e-09 ...,   1.58826922e-04
    2.36975393e-05   2.84606911e-04]]


In [ ]:
prob_test=regr.predict_proba(test_dataset)
print(prob_test[13])

In [34]:
df=pd.DataFrame(probabilities,columns=fish_names_train,index=test_images)
print(df.head)


<bound method NDFrame.head of                train\ALB.pickle  train\BET.pickle  train\DOL.pickle  \
img_00005.jpg      1.675802e-11      3.764134e-13      6.791522e-06   
img_00007.jpg      1.000000e+00      3.631072e-15      1.531135e-17   
img_00009.jpg      9.985648e-01      1.357871e-07      1.972550e-15   
img_00018.jpg      9.999948e-01      6.001042e-16      2.514587e-10   
img_00027.jpg      9.999993e-01      1.720385e-08      2.386935e-14   
img_00030.jpg      1.000000e+00      2.994032e-18      8.499030e-18   
img_00040.jpg      3.738771e-03      1.352883e-06      2.548926e-10   
img_00046.jpg      3.364816e-05      1.528706e-12      3.590454e-16   
img_00053.jpg      9.546303e-01      3.823448e-09      1.416663e-11   
img_00071.jpg      9.999980e-01      1.420299e-13      2.034282e-06   
img_00075.jpg      1.000000e+00      1.181772e-15      7.890928e-19   
img_00102.jpg      5.302532e-07      3.344182e-07      2.451693e-08   
img_00103.jpg      2.221292e-04      3.501746e-

In [35]:
df.to_csv('submisstion_7Jan_logisticTF.csv')

In [ ]:
pickle_file = 'submisstion_12Dec_model_fbgs.pickle'

try:
  f = open(pickle_file, 'wb')
  pickle.dump(regr, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise
  